# Data Manipulation

I want to get comfortable with loading and manipulating data in Python. While most of the practice projects will come with structured data which we can load into a `DataFrame` using `pandas`, this one focuses on semi-structured data and how we can "flatten" it and then load it into other formats. The objective is to see how data flows in Python from one object to another and what advantages and disadvantages each offers.

Let's read the `books.json` data set and display the first item in it.

In [1]:
import json
with open('books.json', encoding = 'utf-8') as f:
    books_dict = json.load(f) #load the json file

from pprint import pprint
pprint(books_dict[0]) # print information for the first book
number_of_books = len(books_dict)

{'_id': 1,
 'authors': ['W. Frank Ableson', 'Charlie Collins', 'Robi Sen'],
 'categories': ['Open Source', 'Mobile'],
 'isbn': '1933988673',
 'longDescription': 'Android is an open source mobile phone platform based on '
                    'the Linux operating system and developed by the Open '
                    'Handset Alliance, a consortium of over 30 hardware, '
                    'software and telecom companies that focus on open '
                    'standards for mobile devices. Led by search giant, '
                    'Google, Android is designed to deliver a better and more '
                    'open and cost effective mobile experience.    Unlocking '
                    "Android: A Developer's Guide provides concise, hands-on "
                    'instruction for the Android operating system and '
                    'development tools. This book teaches important '
                    'architectural concepts in a straightforward writing style '
                    

1. The following program will through the entire data and extracts the following information:  

  - title of the book
  - name of the first author
  - name of the second author (if book has more than one author)
  - number of authors
  - ISBN
  - if the word "data" is in the book's description
  - the number of words in the book's description
  - the year the book was published

  Of course because JSON data doesn't necessarily enforce any sort of schema, we can't be sure that the information we are trying to extract exists for every book. For example, if the book only has one author, then there is no second author. So use `try` and `except` as we loop through every book and skip to the next item every time some information is missing.

  Store the extracted data in a list named `rows` whose elements are tuples, one tuple per book. For example, the first element of `rows` stores the tuple for the first book and should look like this:

        ('Unlocking Android', 'W. Frank Ableson', 'Charlie Collins', 3, '1933988673', True, 252, 2009)

In [2]:
# This function is to extract information from a python dictionary and return the followiing information: title,
# first author, second author, number of authors,ISBN, if the description contains the word "data", description word count, and published year
# JSON data doesn't necessarily enforce any sort of schema, I use try and except as I loop through every book

import datetime
def extract_info(some_dict):
    try:
        title = some_dict['title']
    except:
        title = None
    try:
        first_author = some_dict['authors'][0] 
    except:
        first_author = None
    try:
        number_of_authors = len(some_dict['authors'])
    except:
        number_of_authors = 0
    try:
        second_author = some_dict['authors'][1]
    except:
        second_author = None
    try:
        ISBN = some_dict['isbn']      
    except:
        ISBN = None
    try:
        description_word_count = len(some_dict['longDescription'].split())
        if 'data' in some_dict['longDescription'].split():
            has_data = "True"
        else: 
            has_data = "False"
    except:
        description_word_count = 0
        has_data = "False"
    try:     
        published_year = datetime.datetime.strptime(some_dict['publishedDate']['$date'], '%Y-%m-%dT%H:%M:%S.%f%z').year
    except:
        published_year = None
    return (title,first_author, second_author, number_of_authors,ISBN, has_data, description_word_count,published_year)
    
all_books_info=[0 for i in range(len(books_dict))] # create an array to store info
for i in range(len(books_dict)):
    all_books_info[i] = extract_info(books_dict[i]) # call extract info function and store into each element
    print(all_books_info[i]) #print for result visibility 

('Unlocking Android', 'W. Frank Ableson', 'Charlie Collins', 3, '1933988673', 'True', 252, 2009)
('Android in Action, Second Edition', 'W. Frank Ableson', 'Robi Sen', 2, '1935182722', 'False', 101, 2011)
('Specification by Example', 'Gojko Adzic', None, 1, '1617290084', 'False', 0, 2011)
('Flex 3 in Action', 'Tariq Ahmed with Jon Hirschi', 'Faisal Abid', 2, '1933988746', 'True', 254, 2009)
('Flex 4 in Action', 'Tariq Ahmed', 'Dan Orlando', 4, '1935182420', 'True', 329, 2010)
('Collective Intelligence in Action', 'Satnam Alag', None, 1, '1933988312', 'True', 244, 2008)
('Zend Framework in Action', 'Rob Allen', 'Nick Lo', 3, '1933988320', 'False', 291, 2008)
('Flex on Java', 'Bernerd Allmon', 'Jeremy Anderson', 2, '1933988797', 'False', 273, 2010)
('Griffon in Action', 'Andres Almiray', 'Danno Ferrin', 4, '1935182234', 'False', 240, 2012)
('OSGi in Depth', 'Alexandre de Castro Alves', None, 1, '193518217X', 'False', 269, 2011)
('Flexible Rails', 'Peter Armstrong', None, 1, '1933988509', 

2. Save the content of `rows` in a SQL-like table using `sqlite3`, and choose the appropriate column types. 

  As your column names use the following:

  - `title`
  - `author_1`
  - `author_2`
  - `num_authors`
  - `isbn`
  - `has_data`
  - `desc_len`
  - `year_published`

In [3]:
import sqlite3

connection = sqlite3.connect(':memory:')
cursor = connection.cursor()

cursor.execute('''CREATE TABLE books
             (title text, author_1 text, author_2 text, num_author int, isbn text, has_data text, desc_len int, year_published int) ''')
cursor.executemany('INSERT INTO books VALUES (?,?,?,?,?,?,?,?)',all_books_info)


3. Write a SQL query against the table to show all books that (1) contain the word "data" and (2) have more than 3 authors. Store the result of the query in an object called `books_table`, then close the connection. 

In [4]:
books_table = []
for row in connection.cursor().execute('SELECT * FROM books where has_data = "True" AND num_author >= 3;'):
    books_table.append(row)
    print(row)
books_table
connection.commit()
connection.close()

('Unlocking Android', 'W. Frank Ableson', 'Charlie Collins', 3, '1933988673', 'True', 252, 2009)
('Flex 4 in Action', 'Tariq Ahmed', 'Dan Orlando', 4, '1935182420', 'True', 329, 2010)
('iBATIS in Action', 'Clinton Begin', 'Brandon Goodin', 3, '1932394826', 'True', 159, 2007)
('Spring Dynamic Modules in Action', 'Arnaud Cogoluegnes', 'Thierry Templier', 4, '1935182307', 'True', 183, 2010)
('Portlets and Apache Portals', 'Stefan Hepper', 'Peter Fischer', 5, None, 'True', 153, 2005)
('Struts in Action', 'Ted N. Husted', 'Cedric Dumoulin', 4, '1932394249', 'True', 137, 2002)
('Groovy in Action', 'Dierk Koenig with Andrew Glover', 'Paul King', 4, '1932394842', 'True', 235, 2007)
('Groovy in Action, Second Edition', 'Dierk König', 'Guillaume Laforge', 8, '1935182447', 'True', 289, None)
('Tuscany SCA in Action', 'Simon Laws', 'Mark Combellack', 5, '1933988894', 'True', 245, 2011)
('LINQ in Action', 'Fabrice Marguerie', 'Steve Eichert', 3, '1933988169', 'True', 329, 2008)
('Entity Framework 4

SQL tables are not the only way, and definitely not the most straightforward way to store and manipulate data in Python. A format that's more popular with data scientist is to use the `pandas` library to create a `DataFrame`. This library has a lot of functionality that makes it easy to run the common tasks data scientists do with data.

4. Read the data from the above query into a `DataFrame` and call it `books_df`. HINT: Use `pd.DataFrame` and specify the names to use for the columns.

In [6]:
import pandas as pd

books_df = pd.DataFrame(data = books_table, columns = ['title','author_1','author_2','num_authors','ISBN','has_data','desc_len','year_published'])
books_df


,title,author_1,author_2,num_authors,ISBN,has_data,desc_len,year_published
0,Unlocking Android,W. Frank Ableson,Charlie Collins,3,1933988673,True,252,2009.0
1,Flex 4 in Action,Tariq Ahmed,Dan Orlando,4,1935182420,True,329,2010.0
2,iBATIS in Action,Clinton Begin,Brandon Goodin,3,1932394826,True,159,2007.0
3,Spring Dynamic Modules in Action,Arnaud Cogoluegnes,Thierry Templier,4,1935182307,True,183,2010.0
4,Portlets and Apache Portals,Stefan Hepper,Peter Fischer,5,None,True,153,2005.0
5,Struts in Action,Ted N. Husted,Cedric Dumoulin,4,1932394249,True,137,2002.0
6,Groovy in Action,Dierk Koenig with Andrew Glover,Paul King,4,1932394842,True,235,2007.0
7,"Groovy in Action, Second Edition",Dierk König,Guillaume Laforge,8,1935182447,True,289,NaN
8,Tuscany SCA in Action,Simon Laws,Mark Combellack,5,1933988894,True,245,2011.0
9,LINQ in Action,Fabrice Marguerie,Steve Eichert,3,1933988169,True,329,2008.0


5. Display the first few columns of a `DataFrame` by calling its `head` method. 

In [7]:
books_df[books_df.columns[0:2]].head()

,title,author_1
0,Unlocking Android,W. Frank Ableson
1,Flex 4 in Action,Tariq Ahmed
2,iBATIS in Action,Clinton Begin
3,Spring Dynamic Modules in Action,Arnaud Cogoluegnes
4,Portlets and Apache Portals,Stefan Hepper


Another way of saying it is that a `DataFrame` is an **abstraction** on top of `numpy` arrays: i.e. a `DataFrame` is a more **high-level** object than a `numpy` array. 

6. Call the `values` attribute of your `DataFrame` to convert it into a numpy array and display the first 3 elements of the array.

In [10]:
np_array_books_df = books_df.values
np_array_books_df[0:3]

array([['Unlocking Android', 'W. Frank Ableson', 'Charlie Collins', 3,
        '1933988673', 'True', 252, 2009.0],
       ['Flex 4 in Action', 'Tariq Ahmed', 'Dan Orlando', 4,
        '1935182420', 'True', 329, 2010.0],
       ['iBATIS in Action', 'Clinton Begin', 'Brandon Goodin', 3,
        '1932394826', 'True', 159, 2007.0]], dtype=object)

Now we can judge which object is more "user-friendly". That's one of the things that abstractions allow us to do: build more user-friendly (abstract) objects from less user-friendly (but more fundamental) objects.
